In [98]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset
import pandas as pd
import numpy as np

subscription_id = 'caaae8aa-a0a7-44db-a52c-2690c23ed8c4'
resource_group = 'ST2AIC-BD-1-SG1'
workspace_name = 'SAHNOU_WORKSPACE'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='house-price')
df = dataset.to_pandas_dataframe()

df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
1,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,FR2,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500
2,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,Inside,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500
3,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,Corner,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
4,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,FR2,...,0,NA,NA,NA,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62,7917,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,NA,NA,NA,0,8,2007,WD,Normal,175000
1456,20,RL,85,13175,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,NA,MnPrv,NA,0,2,2010,WD,Normal,210000
1457,70,RL,66,9042,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,NA,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,20,RL,68,9717,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,NA,NA,NA,0,4,2010,WD,Normal,142125


In [99]:
print("Number of columns: " + str (df.shape[1]))
print("number of rows: " + str (df.shape[0]))

Number of columns: 80
number of rows: 1460


In [100]:
df['SalePrice'].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [101]:
df = df.replace('NA', np.NaN)

df_data = df.loc[:, 'MSSubClass': 'SaleCondition']
df_data_na = (df_data.isnull().sum()/ len(df_data)) * 100
df_data_na = df_data_na.drop(df_data_na[df_data_na == 0].index).sort_values(ascending = False)

missing_data = pd.DataFrame({'Missing Ratio':df_data_na})
missing_data

,Missing Ratio
PoolQC,99.520548
MiscFeature,96.301370
Alley,93.767123
Fence,80.753425
FireplaceQu,47.260274
LotFrontage,17.739726
GarageYrBlt,5.547945
GarageType,5.547945
GarageFinish,5.547945
GarageQual,5.547945


## Data cleaning

In [102]:
for col in ('PoolQC','MiscFeature','GarageType','Alley','Fence','FireplaceQu','GarageFinish',
           'GarageQual','GarageCond','MasVnrType','MSSubClass', 'BsmtQual', 'BsmtCond',
           'BsmtExposure','BsmtFinType1','BsmtFinType2'):
    df[col] = df[col].fillna('None')

for col in ('GarageYrBlt','GarageArea', 'GarageCars'):
    df[col] = df[col].fillna(0)
    
for col in ('BsmtFinSF1','BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF','BsmtFullBath',
            'BsmtHalfBath', 'MasVnrArea'):
    df[col] = df[col].fillna(0)
    
df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(
lambda x: x.fillna(x.median()))

df['MSZoning'] = df['MSZoning'].fillna(df['MSZoning'].mode()[0])

df["Functional"] = df["Functional"].fillna('Typ')

df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])

df['KitchenQual'] = df['KitchenQual'].fillna(df['KitchenQual'].mode()[0])

df['Exterior1st']= df['Exterior1st'].fillna(df['Exterior1st'].mode()[0])
df['Exterior2nd']= df['Exterior2nd'].fillna(df['Exterior2nd'].mode()[0])

df['SaleType'] = df['SaleType'].fillna(df['SaleType'].mode()[0])

In [103]:
df = df.drop(['Utilities'], axis=1) #We can delete Utilities as it is full of AllPub values

In [104]:
#Transforming required numerical features to categorical 
df['MSSubClass']= df['MSSubClass'].apply(str)
df['OverallCond'] = df['OverallCond'].astype(str)
df['YrSold'] = df['YrSold'].astype(str)
df['MoSold'] = df['MoSold'].astype(str)

In [105]:
#Label Encoding some categorical variables
#for information in their ordering set

from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

#apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder()
    lbl.fit(list(df[c].values))
    df[c] = lbl.transform(list(df[c].values))
#shape
print('Shape all_data: {}'.format(df.shape))

Shape all_data: (1460, 79)


In [89]:
#add total surface area as TotalSf = basement + firstflr + secondflr
df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']

In [90]:
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,TotalSF
0,9,RL,65,8450,1,1,3,Lvl,Inside,0,...,3,4,None,0,4,2,WD,Normal,208500,2566
1,4,RL,80,9600,1,1,3,Lvl,FR2,0,...,3,4,None,0,7,1,WD,Normal,181500,2524
2,9,RL,68,11250,1,1,0,Lvl,Inside,0,...,3,4,None,0,11,2,WD,Normal,223500,2706
3,10,RL,60,9550,1,1,0,Lvl,Corner,0,...,3,4,None,0,4,0,WD,Abnorml,140000,2473
4,9,RL,84,14260,1,1,0,Lvl,FR2,0,...,3,4,None,0,3,2,WD,Normal,250000,3343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,9,RL,62,7917,1,1,3,Lvl,Inside,0,...,3,4,None,0,10,1,WD,Normal,175000,2600
1456,4,RL,85,13175,1,1,3,Lvl,Inside,0,...,3,2,None,0,4,4,WD,Normal,210000,3615
1457,10,RL,66,9042,1,1,3,Lvl,Inside,0,...,3,0,Shed,2500,7,4,WD,Normal,266500,3492
1458,4,RL,68,9717,1,1,3,Lvl,Inside,0,...,3,4,None,0,6,4,WD,Normal,142125,2156


In [97]:
df = pd.get_dummies(df)
print(df.shape)

(1460, 447)
